In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, activations, initializers, losses, optimizers, metrics

import os
tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # https://stackoverflow.com/questions/35911252/disable-tensorflow-debugging-information

<br> 

#### 1. Prepare dataset & train the model

In [2]:
(train_data, train_label), (test_data, test_label) = datasets.mnist.load_data()

train_data = train_data.reshape(60000, 784).astype('float32') / 255.0 # 0~255 -> 0~1
test_data = test_data.reshape(10000, 784).astype('float32') / 255.0 # 0~255 -> 0~1

train_label = utils.to_categorical(train_label) # 0~9 -> one-hot vector
test_label = utils.to_categorical(test_label) # 0~9 -> one-hot vector


model = models.Sequential() 

model.add(layers.Dense(input_dim=28*28, units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu')) # layers.ELU or layers.LeakyReLU
model.add(layers.Dropout(rate=0.2))

model.add(layers.Dense(units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu')) # layers.ELU or layers.LeakyReLU
model.add(layers.Dropout(rate=0.2))

model.add(layers.Dense(units=10, activation='softmax')) # 0~9 

model.compile(optimizer=optimizers.Adam(), 
              loss=losses.categorical_crossentropy, 
              metrics=[metrics.categorical_accuracy]) 

model.fit(train_data, train_label, batch_size=100, epochs=3, validation_split=0.2) 

Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 3s 68us/sample - loss: 0.3059 - categorical_accuracy: 0.9083 - val_loss: 0.1335 - val_categorical_accuracy: 0.9593
Epoch 2/3
48000/48000 [==============================] - 2s 51us/sample - loss: 0.1417 - categorical_accuracy: 0.9569 - val_loss: 0.0984 - val_categorical_accuracy: 0.9710
Epoch 3/3
48000/48000 [==============================] - 2s 51us/sample - loss: 0.1048 - categorical_accuracy: 0.9680 - val_loss: 0.0876 - val_categorical_accuracy: 0.9729


<br> 

#### 2. Test the model before saving

In [3]:
result = model.evaluate(test_data, test_label, batch_size=100)

print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

10000/10000 [==============================] - 0s 16us/sample - loss: 0.0828 - categorical_accuracy: 0.9743
loss (cross-entropy) : 0.08277038647327573
test accuracy : 0.9743


<br> 

#### 3. Save the trained model

In [4]:
# 저장되는 항목 
# - Model의 architecture
# - Model의 모든 weights (Parameter Theta)
# - The state of the optimizer (바로 모델 학습의 재개 가능)

model.save('mnist_2layer_bn.h5') 
# model.save_weights('mnist_2layer_bn_weight.h5') # weights만 따로 저장도 가능함

<br> 

#### 4. Load the saved model 

In [5]:
model = models.load_model('mnist_2layer_bn.h5')
# model.load_weights('mnist_2layer_bn_weight.h5') # weights만 따로 불러올 수 있음

In [6]:
result = model.evaluate(test_data, test_label, batch_size=100)

print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

10000/10000 [==============================] - 0s 44us/sample - loss: 0.0764 - categorical_accuracy: 0.9757
loss (cross-entropy) : 0.07639191619644407
test accuracy : 0.9757
